In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")

In [3]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna()

In [5]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [6]:
df.count()

Timestamp            3613769
Open                 3613769
High                 3613769
Low                  3613769
Close                3613769
Volume_(BTC)         3613769
Volume_(Currency)    3613769
Weighted_Price       3613769
dtype: int64

In [7]:
df = df[["Timestamp", "Close"]]

In [8]:
df.head()

,Timestamp,Close
0,1325317920,4.39
478,1325346600,4.39
547,1325350740,4.57
548,1325350800,4.58
1224,1325391360,4.58


In [9]:
df.dtypes

Timestamp      int64
Close        float64
dtype: object

In [10]:
# timestamp to date and time
df["Date"] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
df["Time"] = pd.to_datetime(df['Timestamp'],unit='s').dt.time

In [11]:
df.head()

,Timestamp,Close,Date,Time
0,1325317920,4.39,2011-12-31,07:52:00
478,1325346600,4.39,2011-12-31,15:50:00
547,1325350740,4.57,2011-12-31,16:59:00
548,1325350800,4.58,2011-12-31,17:00:00
1224,1325391360,4.58,2012-01-01,04:16:00


In [12]:
# delete useless columns
df = df[["Date","Time", "Close"]]
df.head()

,Date,Time,Close
0,2011-12-31,07:52:00,4.39
478,2011-12-31,15:50:00,4.39
547,2011-12-31,16:59:00,4.57
548,2011-12-31,17:00:00,4.58
1224,2012-01-01,04:16:00,4.58


In [13]:
# reset index and show
df =df.reset_index(drop=True)
df.head(10)

,Date,Time,Close
0,2011-12-31,07:52:00,4.39
1,2011-12-31,15:50:00,4.39
2,2011-12-31,16:59:00,4.57
3,2011-12-31,17:00:00,4.58
4,2012-01-01,04:16:00,4.58
5,2012-01-01,15:28:00,4.84
6,2012-01-01,22:45:00,5.00
7,2012-01-02,20:04:00,5.00
8,2012-01-03,11:45:00,5.32
9,2012-01-03,14:22:00,5.14


In [14]:
# get the close value by using the max time of a date 
df_close = df.groupby("Date").max("Time")
df_close.head()

,Close
Date,
2011-12-31,4.58
2012-01-01,5.00
2012-01-02,5.00
2012-01-03,5.32
2012-01-04,5.57


In [15]:
# reset index again
df_close = df_close.reset_index()

In [16]:
df_close.head()

,Date,Close
0,2011-12-31,4.58
1,2012-01-01,5.00
2,2012-01-02,5.00
3,2012-01-03,5.32
4,2012-01-04,5.57


In [17]:
# get date index, for testing purpose
df_close.index[df_close["Date"] == min(df_close["Date"])][0]

0

In [18]:
# moving avarage calculation function
def ma(date, n_days):
    index = df_close.index[df_close["Date"] == date][0]
    df = None
    if index == 0:
        df = df_close.iloc[0]
    elif index - n_days >= 0:
        df = df_close.iloc[index - n_days + 1 : index+1]
    else:
        df = df_close.iloc[0 : index + 1]
    return df["Close"].mean()        
#    return df

In [19]:
# test the function
#from datetime import datetime, timedelta
#d = min(df_close["Date"] + timedelta(days=5))
#print(d)
#ma(d, 5)

In [20]:
# test the function
#ma(d, 5)["Close"].mean()

In [21]:
# make a copy of the df
df_close_ma = df_close.copy()

In [22]:
# add different ma point to each date
ma_days = [5, 10, 20, 30, 60, 180, 240]
for ma_day in ma_days:
    column_name = f"ma_{ma_day}"
    df_close_ma[column_name] = df_close_ma.apply(lambda x: ma(x["Date"], ma_day), axis=1)

In [23]:
df_close_ma

,Date,Close,ma_5,ma_10,ma_20,ma_30,ma_60,ma_180,ma_240
0,2011-12-31,4.58,4.580,4.580,4.5800,4.580000,4.580000,4.580000,4.580000
1,2012-01-01,5.00,4.790,4.790,4.7900,4.790000,4.790000,4.790000,4.790000
2,2012-01-02,5.00,4.860,4.860,4.8600,4.860000,4.860000,4.860000,4.860000
3,2012-01-03,5.32,4.975,4.975,4.9750,4.975000,4.975000,4.975000,4.975000
4,2012-01-04,5.57,5.094,5.094,5.0940,5.094000,5.094000,5.094000,5.094000
...,...,...,...,...,...,...,...,...,...
3371,2021-03-27,56591.90,55574.128,57420.123,57748.1875,55094.464333,50379.645833,30956.614167,26057.659250
3372,2021-03-28,56545.59,55718.246,57068.813,57954.0770,55364.072000,50779.072333,31210.446278,26245.553042
3373,2021-03-29,58309.35,55952.392,56955.536,58122.9400,55696.360667,51186.950500,31474.137111,26438.996500
3374,2021-03-30,59371.86,57183.586,56907.024,58224.6975,56121.904667,51532.865000,31743.341889,26635.989708
